In [2]:
import os
import psycopg
import pandas as pd
from catboost import CatBoostRegressor
import joblib
import json
from sklearn.model_selection import KFold, cross_validate
import mlflow
import yaml
from mlflow.client import MlflowClient

/home/mle-user/mle_projects/yn/.y3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/mle_projects/yn/.y3/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [9]:

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'old_model'
RUN_NAME = "baseline+data"
REGISTRY_MODEL_NAME = 'baseline'
FS_ASSETS = "baseline"  

In [10]:
# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

In [11]:
mlflow.set_experiment(EXPERIMENT_NAME)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
with mlflow.start_run(run_name=f"{RUN_NAME}_intersection_and_union", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    with open('cv_results/cv_res.json', 'r') as fd:
        cv_results = yaml.safe_load(fd)
    model = joblib.load('models/fitted_model.pkl')
    fit_params = model.get_params()
    mlflow.log_params(fit_params)
    mlflow.log_metrics(cv_results)
    
    data = pd.read_csv('data/initial_data.csv')
    mlflow.log_artifacts('data')
    X = data.drop(['price'], axis=1)
    metadata = {"model_type": "baseline", 'structure': 'Catboost'}
    train_data = X.head(5)
    pred = model.predict(train_data)
    signature = mlflow.models.infer_signature(train_data, pred)
    input_example = train_data
    pip_requirements = '/home/mle-user/mle_projects/yn/requirements.txt'
    model_info = mlflow.catboost.log_model( 
		    cb_model=model,
            artifact_path="models",
            registered_model_name=REGISTRY_MODEL_NAME,
            signature=signature,
            input_example=input_example,
            pip_requirements=pip_requirements,
            metadata = metadata,
            await_registration_for=60
		)
    mlflow.end_run()
    

2025/03/18 09:29:44 INFO mlflow.tracking.fluent: Experiment with name 'old_model' does not exist. Creating a new experiment.
/home/mle-user/mle_projects/yn/.y3/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Successfully registered model 'basel